In [2]:
import pandas as pd
import torch
import igraph as ig
import os
import glob
from torch_geometric.data import Data

# Read the Excel file
df = pd.read_excel('Data_NoTime.xlsx')

# Calculate the maximum values for normalization
#max_time = df['time'].max()
max_temperature = df['temperature'].max()

# Normalize 'time' and 'temperature' using the maximum values
#df['time'] = df['time'] / max_time
df['temperature'] = df['temperature'] / max_temperature

# Create a dictionary to map filenames to their target values
#file_to_targets = {row["image_name"]: [row["Cr"], row["Co"], row["temperature"], row["time"]] for _, row in df.iterrows()}
file_to_targets = {row["image_name"]: [row["Cr"], row["Co"], row["temperature"]] for _, row in df.iterrows()}

# Define the path to the GraphML files directory and process each GraphML file
graphml_dir_path = "Graph_Generation"
data_list = []

for graphml_file_path in glob.glob(os.path.join(graphml_dir_path, "*.graphml")):
    filename = os.path.basename(graphml_file_path)
    
    if filename in file_to_targets:
        # Load the graph using igraph
        graph = ig.Graph.Read_GraphML(graphml_file_path)

        # Extract node features
        node_features = [[vertex['value']] for vertex in graph.vs]
        edge_indices = [[edge.source, edge.target] for edge in graph.es]
        edge_attrs = [[edge['weight']] for edge in graph.es]

        # Convert to PyTorch tensors
        x = torch.tensor(node_features, dtype=torch.float)
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

        # Get the target values for this graph
        target = torch.tensor(file_to_targets[filename], dtype=torch.float32)

        # Reshape target to [1, 4] (assuming each graph has exactly one target)
        #target_reshaped = target.view(1, 3)
        target_reshaped = target.view(-1, 3)  # Make sure the target has the shape [batch_size, 3]


        # Create PyTorch Geometric Data object
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=target_reshaped)
        #print("data.x",data.x)
        data_list.append(data)

# Print the number of Data objects created
print(f"Number of Data objects created: {len(data_list)}")


Number of Data objects created: 386


In [3]:
df.head()

,Cr,Co,temperature,image_name
0,0.45,0.35,0.886598,cr0.45_co0.35_T860_360000.graphml
1,0.45,0.35,0.896907,cr0.45_co0.35_T870_360000.graphml
2,0.45,0.25,0.927835,cr0.45_co0.25_T900_360000.graphml
3,0.45,0.35,0.907216,cr0.45_co0.35_T880_360000.graphml
4,0.45,0.25,0.938144,cr0.45_co0.25_T910_360000.graphml


In [4]:
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader

# Split data_list into training and testing sets
train_data, test_data = train_test_split(data_list, test_size=0.2, random_state=42)

# Create DataLoader instances for training and testing data
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Print the number of training and testing samples
print(f"Number of training samples: {len(train_data)}")
print(f"Number of testing samples: {len(test_data)}")

Number of training samples: 308
Number of testing samples: 78


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool

# Define the GraphSAGE model with weight initialization
class GraphSAGEModel(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=128):
        super(GraphSAGEModel, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels, aggr='mean')
        self.conv2 = SAGEConv(hidden_channels, hidden_channels, aggr='mean')
        self.conv3 = SAGEConv(hidden_channels, hidden_channels, aggr='mean')
        self.conv4 = SAGEConv(hidden_channels, hidden_channels, aggr='mean')
        self.conv5 = SAGEConv(hidden_channels, hidden_channels, aggr='mean')
        self.conv6 = SAGEConv(hidden_channels, hidden_channels, aggr='mean')
        self.conv7 = SAGEConv(hidden_channels, hidden_channels, aggr='mean')
        self.conv8 = SAGEConv(hidden_channels, hidden_channels, aggr='mean')
        self.fc1 = nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, hidden_channels)
        self.fc3 = nn.Linear(hidden_channels, hidden_channels)
        self.fc4 = nn.Linear(hidden_channels, hidden_channels)
        self.fc5 = nn.Linear(hidden_channels, out_channels)

        # Apply weight initialization
        self.initialize_weights()

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, SAGEConv):
                # Apply Xavier initialization for GraphSAGE layers
                torch.nn.init.xavier_uniform_(m.lin_l.weight)
                torch.nn.init.xavier_uniform_(m.lin_r.weight)
                if m.lin_l.bias is not None:
                    torch.nn.init.zeros_(m.lin_l.bias)
            elif isinstance(m, nn.Linear):
                # Apply He initialization for fully connected layers
                torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)

    def forward(self, x, edge_index, edge_attr, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))
        x = F.relu(self.conv6(x, edge_index))
        x = F.relu(self.conv7(x, edge_index))
        x = F.relu(self.conv8(x, edge_index))
        x = global_mean_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)  # No activation in the final layer
        return x

# Instantiate the model
model = GraphSAGEModel(in_channels=1, out_channels=3)

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.L1Loss()  # Mean Absolute Error


In [6]:
import torch
import torch.nn as nn
import csv

def train(loader):
    model.train()
    total_loss = 0
    train_predictions = []
    train_targets = []

    for batch_idx, batch in enumerate(loader):
        batch.x.requires_grad = True
        
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)

        loss = criterion(out, batch.y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        total_loss += loss.item()
        train_predictions.append(out)
        train_targets.append(batch.y)

    predictions = torch.cat(train_predictions, dim=0)
    targets = torch.cat(train_targets, dim=0)

    return total_loss / len(loader), predictions, targets


def test(loader):
    model.eval()
    total_loss = 0
    test_predictions = []
    test_targets = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(loader):
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)

            loss = criterion(out, batch.y)
            total_loss += loss.item()
            test_predictions.append(out)
            test_targets.append(batch.y)

    predictions = torch.cat(test_predictions, dim=0)
    targets = torch.cat(test_targets, dim=0)

    return total_loss / len(loader), predictions, targets


# Logging and plotting data
train_losses = []
test_losses = []

# Training loop with testing
num_epochs = 50
for epoch in range(num_epochs):
    # Training
    train_loss, train_predictions, train_targets = train(train_data)
    train_losses.append(train_loss)

    # Extract individual predictions and true values for Cr, Co, and temperature
    pred_Cr_train, pred_Co_train, pred_temperature_train = train_predictions[:, 0], train_predictions[:, 1], train_predictions[:, 2]
    true_Cr_train, true_Co_train, true_temperature_train = train_targets[:, 0], train_targets[:, 1], train_targets[:, 2]

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

    # Testing
    test_loss, test_predictions, test_targets = test(test_data)
    test_losses.append(test_loss)

    pred_Cr_test, pred_Co_test, pred_temperature_test = test_predictions[:, 0], test_predictions[:, 1], test_predictions[:, 2]
    true_Cr_test, true_Co_test, true_temperature_test = test_targets[:, 0], test_targets[:, 1], test_targets[:, 2]

# Save train and test losses to a CSV file
with open('losses.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Train Loss', 'Test Loss'])
    for epoch in range(num_epochs):
        writer.writerow([epoch+1, train_losses[epoch], test_losses[epoch]])


Epoch 1/50, Train Loss: 0.1052
Epoch 2/50, Train Loss: 0.0768
Epoch 3/50, Train Loss: 0.0757
Epoch 4/50, Train Loss: 0.0733
Epoch 5/50, Train Loss: 0.0711
Epoch 6/50, Train Loss: 0.0698
Epoch 7/50, Train Loss: 0.0680
Epoch 8/50, Train Loss: 0.0679
Epoch 9/50, Train Loss: 0.0662
Epoch 10/50, Train Loss: 0.0666
Epoch 11/50, Train Loss: 0.0657
Epoch 12/50, Train Loss: 0.0644
Epoch 13/50, Train Loss: 0.0641
Epoch 14/50, Train Loss: 0.0626
Epoch 15/50, Train Loss: 0.0629
Epoch 16/50, Train Loss: 0.0610
Epoch 17/50, Train Loss: 0.0607
Epoch 18/50, Train Loss: 0.0610
Epoch 19/50, Train Loss: 0.0608
Epoch 20/50, Train Loss: 0.0596
Epoch 21/50, Train Loss: 0.0589
Epoch 22/50, Train Loss: 0.0577
Epoch 23/50, Train Loss: 0.0600
Epoch 24/50, Train Loss: 0.0565
Epoch 25/50, Train Loss: 0.0566
Epoch 26/50, Train Loss: 0.0572
Epoch 27/50, Train Loss: 0.0588
Epoch 28/50, Train Loss: 0.0556
Epoch 29/50, Train Loss: 0.0553
Epoch 30/50, Train Loss: 0.0529
Epoch 31/50, Train Loss: 0.0499
Epoch 32/50, Trai

In [7]:
# Print first 10 values for true and predicted time and temperature
print("First 10 true and predicted values for Time and Temperature:")
for i in range(10):
    #print(f"True Time: {true_time[i].item():.3f}, Pred Time: {pred_time[i].item():.3f}")
    print(f"True Temperature: {true_temperature_train[i].item():.3f}, Pred Temperature: {pred_temperature_train[i].item():.3f}")
    

First 10 true and predicted values for Time and Temperature:
True Temperature: 0.979, Pred Temperature: 0.987
True Temperature: 0.959, Pred Temperature: 0.970
True Temperature: 0.959, Pred Temperature: 0.970
True Temperature: 0.918, Pred Temperature: 0.926
True Temperature: 0.990, Pred Temperature: 0.991
True Temperature: 0.959, Pred Temperature: 0.985
True Temperature: 0.959, Pred Temperature: 0.969
True Temperature: 0.938, Pred Temperature: 0.936
True Temperature: 0.969, Pred Temperature: 0.963
True Temperature: 0.887, Pred Temperature: 0.861


In [8]:
print("First 10 true and predicted values for Cr and Co:")
for i in range(10):
    print(f"True Cr: {true_Cr_train[i].item():.3f}, Pred Cr: {pred_Cr_train[i].item():.3f}")
    #print(f"True Co: {true_Co[i].item():.3f}, Pred Co: {pred_Co[i].item():.3f}")
    

First 10 true and predicted values for Cr and Co:
True Cr: 0.300, Pred Cr: 0.271
True Cr: 0.400, Pred Cr: 0.386
True Cr: 0.450, Pred Cr: 0.448
True Cr: 0.500, Pred Cr: 0.472
True Cr: 0.350, Pred Cr: 0.309
True Cr: 0.300, Pred Cr: 0.265
True Cr: 0.400, Pred Cr: 0.371
True Cr: 0.400, Pred Cr: 0.339
True Cr: 0.400, Pred Cr: 0.394
True Cr: 0.400, Pred Cr: 0.392


In [9]:
print("First 10 true and predicted values for Cr and Co:")
for i in range(10):
    #print(f"True Cr: {true_Cr[i].item():.3f}, Pred Cr: {pred_Cr[i].item():.3f}")
    print(f"True Co: {true_Co_train[i].item():.3f}, Pred Co: {pred_Co_train[i].item():.3f}")

First 10 true and predicted values for Cr and Co:
True Co: 0.250, Pred Co: 0.217
True Co: 0.350, Pred Co: 0.312
True Co: 0.300, Pred Co: 0.251
True Co: 0.250, Pred Co: 0.230
True Co: 0.400, Pred Co: 0.477
True Co: 0.200, Pred Co: 0.289
True Co: 0.150, Pred Co: 0.209
True Co: 0.300, Pred Co: 0.499
True Co: 0.150, Pred Co: 0.197
True Co: 0.450, Pred Co: 0.482


In [10]:
# Denormalize temperature values
true_temp_denorm = true_temperature_train * max_temperature
pred_temp_denorm = pred_temperature_train * max_temperature
print("First 10 denormalized true and predicted values for Temperature:")
for i in range(10):
    print(f"True Temp: {true_temp_denorm[i]:.2f}, Pred Temp: {pred_temp_denorm[i]:.2f}")


First 10 denormalized true and predicted values for Temperature:
True Temp: 950.00, Pred Temp: 957.54
True Temp: 930.00, Pred Temp: 941.23
True Temp: 930.00, Pred Temp: 941.11
True Temp: 890.00, Pred Temp: 898.33
True Temp: 960.00, Pred Temp: 961.57
True Temp: 930.00, Pred Temp: 955.77
True Temp: 930.00, Pred Temp: 939.92
True Temp: 910.00, Pred Temp: 907.57
True Temp: 940.00, Pred Temp: 934.50
True Temp: 860.00, Pred Temp: 834.75


In [11]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Extract individual predictions and true values for Cr, Co, and temperature
pred_Cr_train, pred_Co_train, pred_temperature_train = train_predictions[:, 0], train_predictions[:, 1], train_predictions[:, 2]
true_Cr_train, true_Co_train, true_temperature_train = train_targets[:, 0], train_targets[:, 1], train_targets[:, 2]


def parity_plot(ground_truth, prediction,
                lims_min, lims_max,
                plot_name, r2_value=None,
                ground_truth_norm=None, prediction_norm=None):
    # plt.figure()
    plt.figure(figsize=(3.5, 3.5), dpi=200)

    # Scatter plot of true vs predicted values
    plt.scatter(ground_truth, prediction,  color='black', s=8)
    
    # Define the line for perfect prediction
    plt.plot([lims_min, lims_max], [lims_min, lims_max], 'k--',linewidth=1)
    
    # Set limits for x and y axes to the range of your data
    plt.xlim(lims_min, lims_max)  # X-axis limit
    plt.ylim(lims_min, lims_max)  # Y-axis limit
    
    # Add labels, title, legend, and grid
    plt.xlabel('Ground Truth', fontsize=20, labelpad=4)
    plt.ylabel('Prediction', fontsize=20, labelpad=4)
    plt.title(plot_name, fontsize=20,loc='center')
    plt.legend().remove()
    plt.grid(False)
    plt.tick_params(axis='both', which='both', length=0)
    
    # Calculate MSE
    # mse = mean_squared_error(ground_truth, prediction)
    # Use normalized values if provided, otherwise fall back to original values
    gt_mse = ground_truth_norm if ground_truth_norm is not None else ground_truth
    pred_mse = prediction_norm if prediction_norm is not None else prediction
    mse = mean_squared_error(gt_mse, pred_mse)
#     # Add R² and MSE as text to the plot
#     plt.text(
#         0.05, 0.95, 
#         f"R-Squared = {r2_value:.3f}\nMSE = {mse:.2E}", 
#         transform=plt.gca().transAxes,
#         fontsize=10,
#         verticalalignment='top')
    
#     # Show the plot
#     #plt.show()
#     plt.savefig(file_path)
#     plt.close()
    # Display normalized MSE and R²
    plt.text(
        0.02, 0.98,
        f'$\\bf{{R-Squared}}$ = {r2_value:.4f}\n$\\bf{{MSE}}$ = {mse:.1E}',
        fontsize=20,
        color='black',
        ha='left',
        va='top',
        transform=plt.gca().transAxes
    ) 
    plt.tight_layout(pad=0.5)
    plt.savefig(plot_name,dpi=150, facecolor='w', edgecolor='w')
    plt.close()

    # Example usage
# true_temperature = true_temperature_train.numpy()
# pred_temperature = pred_temperature_train.detach().numpy()  # Detach the tensor before calling .numpy()

# Normalized values (already between 0 and 1)
true_temp_norm = true_temperature_train.numpy()
pred_temp_norm = pred_temperature_train.detach().numpy()

# Denormalized values
true_temp_denorm = true_temp_norm * max_temperature
pred_temp_denorm = pred_temp_norm * max_temperature

# Calculate R²
r2_temperature = r2_score(true_temp_norm, pred_temp_norm)

# Define plot limits based on your data range
lims_min = min(true_temp_denorm.min(), pred_temp_denorm.min())
lims_max = max(true_temp_denorm.max(), pred_temp_denorm.max())


# Call the function
# parity_plot(true_temperature, pred_temperature, lims_min, lims_max,  "predicted_temperature.png",r2_value=r2_temperature)
parity_plot(
    ground_truth=true_temp_denorm,
    prediction=pred_temp_denorm,
    lims_min=lims_min, lims_max=lims_max,
    plot_name="temerpature_train.png",
    r2_value=r2_temperature,
    ground_truth_norm=true_temp_norm,
    prediction_norm=pred_temp_norm
)


pred_Cr = pred_Cr_train.detach().numpy()
true_Cr = true_Cr_train.numpy()

# Calculate R²
r2_Cr = r2_score(true_Cr, pred_Cr)

# Call the function
# parity_plot(true_Cr, pred_Cr, 0, 1,'parity_plot_Cr.png',r2_value=r2_Cr)
parity_plot(
    ground_truth=true_Cr,
    prediction=pred_Cr,
    lims_min=0, lims_max=1,
    plot_name="Cr_train",
    r2_value=r2_score(true_Cr, pred_Cr)
)
pred_Co = pred_Co_train.detach().numpy()
true_Co = true_Co_train.numpy()

# Calculate R²
r2_Co = r2_score(true_Co, pred_Co)


# Call the function
# parity_plot(true_Co, pred_Co, 0, 1, 'parity_plot_Co.png',r2_value=r2_Co)
parity_plot(
    ground_truth=true_Co,
    prediction=pred_Co,
    lims_min=0, lims_max=1,
    plot_name="Co_train",
    r2_value=r2_score(true_Co, pred_Co)
)

# Convert to numpy for plotting
#pred_time = pred_time.numpy()
#true_time = true_time.numpy()


# Plot name for Cr component
#plot_name_time = "Parity Plot: True vs. Predicted time"

# Call the function
#parity_plot(true_time, pred_time, lims_min, lims_max, plot_name_time,'parity_plot_time.png')

/tmp/ipykernel_7331/3137613047.py:32: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend().remove()
/tmp/ipykernel_7331/3137613047.py:32: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend().remove()
/tmp/ipykernel_7331/3137613047.py:32: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend().remove()


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

pred_Cr_test, pred_Co_test, pred_temperature_test = test_predictions[:, 0], test_predictions[:, 1], test_predictions[:, 2]
true_Cr_test, true_Co_test, true_temperature_test = test_targets[:, 0], test_targets[:, 1], test_targets[:, 2]


def parity_plot(ground_truth, prediction,
                lims_min, lims_max,
                plot_name, r2_value=None,
                ground_truth_norm=None, prediction_norm=None):
    # plt.figure()
    plt.figure(figsize=(3.5,3.5), dpi=200)

    # Scatter plot of true vs predicted values
    plt.scatter(ground_truth, prediction,  color='black', s=5)
    
    # Define the line for perfect prediction
    plt.plot([lims_min, lims_max], [lims_min, lims_max], 'k--',linewidth=1)
    
    # Set limits for x and y axes to the range of your data
    plt.xlim(lims_min, lims_max)  # X-axis limit
    plt.ylim(lims_min, lims_max)  # Y-axis limit
    
    # Add labels, title, legend, and grid
    plt.xlabel('Ground Truth', fontsize=25, labelpad=4)
    plt.ylabel('Prediction', fontsize=25, labelpad=4)
    plt.title(plot_name, fontsize=20,loc='center')
    plt.legend().remove()
    plt.grid(False)
    plt.tick_params(axis='both', which='both', length=0)
    
    # Calculate MSE
    # mse = mean_squared_error(ground_truth, prediction)
    # Use normalized values if provided, otherwise fall back to original values
    gt_mse = ground_truth_norm if ground_truth_norm is not None else ground_truth
    pred_mse = prediction_norm if prediction_norm is not None else prediction
    mse = mean_squared_error(gt_mse, pred_mse)
#     # Add R² and MSE as text to the plot
#     plt.text(
#         0.05, 0.95, 
#         f"R-Squared = {r2_value:.3f}\nMSE = {mse:.2E}", 
#         transform=plt.gca().transAxes,
#         fontsize=10,
#         verticalalignment='top')
    
#     # Show the plot
#     #plt.show()
#     plt.savefig(file_path)
#     plt.close()
    # Display normalized MSE and R²
    plt.text(
        0.02, 0.98,
        f'$\\bf{{R-Squared}}$ = {r2_value:.4f}\n$\\bf{{MSE}}$ = {mse:.1E}',
        fontsize=16,
        color='black',
        ha='left',
        va='top',
        transform=plt.gca().transAxes,
       
    ) 
    plt.tight_layout(pad=0.5)
    plt.savefig(plot_name,dpi=200, facecolor='w', edgecolor='w')
    plt.close()

    # Example usage
# true_temperature = true_temperature_train.numpy()
# pred_temperature = pred_temperature_train.detach().numpy()  # Detach the tensor before calling .numpy()

# Normalized values (already between 0 and 1)
true_temp_norm = true_temperature_test.numpy()
pred_temp_norm = pred_temperature_test.detach().numpy()

# Denormalized values
true_temp_denorm = true_temp_norm * max_temperature
pred_temp_denorm = pred_temp_norm * max_temperature

# Calculate R²
r2_temperature = r2_score(true_temp_norm, pred_temp_norm)

# Define plot limits based on your data range
lims_min = min(true_temp_denorm.min(), pred_temp_denorm.min())
lims_max = max(true_temp_denorm.max(), pred_temp_denorm.max())

# Call the function
# parity_plot(true_temperature, pred_temperature, lims_min, lims_max,  "predicted_temperature.png",r2_value=r2_temperature)
parity_plot(
    ground_truth=true_temp_denorm,
    prediction=pred_temp_denorm,
    lims_min=lims_min, lims_max=lims_max,
    plot_name="temperature",
    r2_value=r2_temperature,
    ground_truth_norm=true_temp_norm,
    prediction_norm=pred_temp_norm
)


pred_Cr = pred_Cr_test.detach().numpy()
true_Cr = true_Cr_test.numpy()

# Calculate R²
r2_Cr = r2_score(true_Cr, pred_Cr)

# Call the function
# parity_plot(true_Cr, pred_Cr, 0, 1,'parity_plot_Cr.png',r2_value=r2_Cr)
parity_plot(
    ground_truth=true_Cr,
    prediction=pred_Cr,
    lims_min=0, lims_max=1,
    plot_name="Cr",
    r2_value=r2_score(true_Cr, pred_Cr)
)
pred_Co = pred_Co_test.detach().numpy()
true_Co = true_Co_test.numpy()

# Calculate R²
r2_Co = r2_score(true_Co, pred_Co)

# Call the function
# parity_plot(true_Co, pred_Co, 0, 1, 'parity_plot_Co.png',r2_value=r2_Co)
parity_plot(
    ground_truth=true_Co,
    prediction=pred_Co,
    lims_min=0, lims_max=1,
    plot_name="Co",
    r2_value=r2_score(true_Co, pred_Co)
)

# Convert to numpy for plotting
#pred_time = pred_time.numpy()
#true_time = true_time.numpy()


# Plot name for Cr component
#plot_name_time = "Parity Plot: True vs. Predicted time"

# Call the function
#parity_plot(true_time, pred_time, lims_min, lims_max, plot_name_time,'parity_plot_time.png')

/tmp/ipykernel_7331/2506608229.py:31: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend().remove()
/tmp/ipykernel_7331/2506608229.py:31: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend().remove()
/tmp/ipykernel_7331/2506608229.py:31: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend().remove()


In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load data
data = pd.read_csv('losses.csv')

# Output directory
os.makedirs('learning_curves', exist_ok=True)

# Create the plot
plt.figure(figsize=(5, 3), dpi=300)

# Plot with smooth, professional lines
plt.plot(data['Epoch'], data['Train Loss'], label='Train loss', color='red', linewidth=2)
plt.plot(data['Epoch'], data['Test Loss'], label='Validation loss', color='green', linestyle='--', linewidth=2)

# Axis labels
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)

# Ticks and formatting
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tick_params(axis='both', which='both', direction='out', length=4)

# Optional: fix y-scale for visual consistency
#plt.ylim(bottom=0, top=100)  # Use if you want to stretch the vertical axis

# Legend
plt.legend(fontsize=12, loc='upper right', frameon=False)

# Clean layout
plt.tight_layout()

# Save as high-resolution PNG (or PDF)
plt.savefig('learning_curves/gnn_learning_curve_pubready.png', dpi=300)
# plt.savefig('learning_curves/gnn_learning_curve_pubready.pdf', format='pdf')

plt.close()


In [14]:
# Print first 10 values for true and predicted time and temperature
print("First 10 test true and predicted values for Time and Temperature:")
for i in range(10):
    #print(f"True Time: {true_time[i].item():.3f}, Pred Time: {pred_time[i].item():.3f}")
    print(f"True Temperature: {true_temperature_test[i].item():.3f}, Pred Temperature: {pred_temperature_test[i].item():.3f}")
    

First 10 test true and predicted values for Time and Temperature:
True Temperature: 1.000, Pred Temperature: 0.993
True Temperature: 0.928, Pred Temperature: 0.946
True Temperature: 0.887, Pred Temperature: 0.933
True Temperature: 0.948, Pred Temperature: 0.966
True Temperature: 0.990, Pred Temperature: 0.979
True Temperature: 0.897, Pred Temperature: 0.894
True Temperature: 0.907, Pred Temperature: 0.917
True Temperature: 0.938, Pred Temperature: 0.951
True Temperature: 0.928, Pred Temperature: 0.931
True Temperature: 0.928, Pred Temperature: 0.954


In [15]:
print("First 10  test true and predicted values for Cr and Co:")
for i in range(10):
    print(f"True Cr: {true_Cr_test[i].item():.3f}, Pred Cr: {pred_Cr_test[i].item():.3f}")
    #print(f"True Co: {true_Co[i].item():.3f}, Pred Co: {pred_Co[i].item():.3f}")
    

First 10  test true and predicted values for Cr and Co:
True Cr: 0.300, Pred Cr: 0.297
True Cr: 0.300, Pred Cr: 0.301
True Cr: 0.450, Pred Cr: 0.463
True Cr: 0.300, Pred Cr: 0.305
True Cr: 0.250, Pred Cr: 0.304
True Cr: 0.400, Pred Cr: 0.440
True Cr: 0.350, Pred Cr: 0.361
True Cr: 0.300, Pred Cr: 0.306
True Cr: 0.450, Pred Cr: 0.411
True Cr: 0.400, Pred Cr: 0.394


In [16]:
print("First 10 test true and predicted values for Cr and Co:")
for i in range(10):
    #print(f"True Cr: {true_Cr[i].item():.3f}, Pred Cr: {pred_Cr[i].item():.3f}")
    print(f"True Co: {true_Co_test[i].item():.3f}, Pred Co: {pred_Co_test[i].item():.3f}")

First 10 test true and predicted values for Cr and Co:
True Co: 0.200, Pred Co: 0.183
True Co: 0.300, Pred Co: 0.331
True Co: 0.100, Pred Co: 0.078
True Co: 0.250, Pred Co: 0.267
True Co: 0.600, Pred Co: 0.462
True Co: 0.450, Pred Co: 0.382
True Co: 0.350, Pred Co: 0.308
True Co: 0.300, Pred Co: 0.309
True Co: 0.400, Pred Co: 0.474
True Co: 0.150, Pred Co: 0.179


In [17]:
import matplotlib.pyplot as plt
import pandas as pd

# Load the data (example assumes you have it in a CSV file or as a DataFrame)
# Replace 'file_path.csv' with your file path
data = pd.read_csv('losses.csv')

# Create output directory if it doesn't exist
os.makedirs('logplot', exist_ok=True)

# Extract values for plotting
epochs = data['Epoch']
train_loss = data['Train Loss']
test_loss = data['Test Loss']

# Create a log plot
plt.figure(figsize=(10, 6))
plt.semilogy(epochs, train_loss, label='Train Loss', color='blue')
plt.semilogy(epochs, test_loss, label='Test Loss', color='red')

# Add labels, title, and legend
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss (Log Scale)', fontsize=14)
plt.title('Training and Testing Loss (Log Scale)', fontsize=16)
plt.legend(fontsize=12)
plt.grid(False, which="both", linestyle='--', linewidth=0.5)

# Save the plot to a PDF file
pdf_file_path = 'logplot/gnn_logplot.png'
plt.savefig(pdf_file_path, format='png')  # Save as PDF

# Show the plot
#plt.show()
plt.close()


/tmp/ipykernel_7331/2253312636.py:26: UserWarning: First parameter to grid() is false, but line properties are supplied. The grid will be enabled.
  plt.grid(False, which="both", linestyle='--', linewidth=0.5)
